# CLOVA Studio - Explorer - 요약
2024.07.17  
- "segMinSize": 300,  
  "includeAiFilters": True,  
  "autoSentenceSplitter": True,  
  "segCount": -1,  
  "segMaxSize": 1000  

In [12]:
## 패키지 설치
import pandas as pd
import json
import http.client

In [13]:
## API
class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/summarization/v2/f54b9441d4cd4cacbcc2a5c914723810', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['text']
        else:
            return 'Error'

In [42]:
## 문장 사이의 기호 처리
import re

def clean_summary(summary):
    # 문장 사이의 특수 문자 제거 및 정리
    summary = re.sub(r'\n\s*', '. ', summary)  # \n 뒤에 있는 공백들을 포함하여 . 으로 바꿈

    # 문장 사이의 특수 문자 제거 및 정리
    summary = re.sub(r'(?<!\w)\s*-\s*', '', summary)  # 문장 앞뒤의 기호 제거
    summary = re.sub(r'\s+', ' ', summary)  # 여러 개의 공백을 하나의 공백으로 대체

    # 문장 시작 전에 띄어쓰기 한 칸 추가
    summary = re.sub(r'(?<=\.|\?|!)\s*', ' ', summary)  # 문장 부호 뒤의 공백을 띄어쓰기로 바꿈

    summary = summary.strip()  # 양쪽 끝의 공백 제거

    # 문장 끝에 마침표가 없는 경우 추가
    if not re.search(r'[.!?]$', summary):
        summary += '.'

    return summary

# 테스트
test_summary = "테슬라 주가를 두고 엇갈린 전망이 나오고 있음\n- 테슬라 주가 강세의 배경으로"
print(clean_summary(test_summary))

테슬라 주가를 두고 엇갈린 전망이 나오고 있음. 테슬라 주가 강세의 배경으로.


In [43]:
## 뉴스요약 함수
def summarize_news_articles(csv_path):
    # CSV 파일 불러오기
    df = pd.read_csv(csv_path, encoding='utf-8')

    # API 연결 설정
    completion_executor = CompletionExecutor(
        host='clovastudio.apigw.ntruss.com',
        api_key='NTA0MjU2MWZlZTcxNDJiY1XHHT1CG2IEYUDfBzRBda5ARQgcLFxGk4gffzMOUc02',
        api_key_primary_val = 'U8NCSe3jtrv3qZxOJxVat54fOq3ztFr34Y5Dgd3q',
        request_id='33edbf3b-7513-4c10-908f-f3b9132b9f63'
    )

    # 뉴스 기사 요약 리스트
    summaries = []

    for index, row in df.iterrows():
        # API 요청 데이터 구성
        request_data = {
            "texts": [row['news_contents']],
            "segMinSize": 300,
            "includeAiFilters": True,
            "autoSentenceSplitter": True,
            "segCount": -1,
            "segMaxSize": 1000
        }

        # API 호출 및 결과 저장
        summary = completion_executor.execute(request_data)
        cleaned_summary = clean_summary(summary)
        summaries.append(cleaned_summary)

    # 요약 결과를 데이터프레임에 추가
    df['summary'] = summaries

    return df

In [44]:
## 데이터 입력 - 출력
if __name__ == '__main__':
    # CSV 파일 경로
    csv_path = "C:/Users/myhyu/OneDrive/Desktop/2024 미래에셋/NewsData/뉴스데이터_0716.csv"
    
    # 뉴스 기사 요약 실행
    summarized_df = summarize_news_articles(csv_path)
    
    # 요약된 데이터프레임 출력
    print(summarized_df[['news_title', 'summary']])
    

                                       news_title  \
0     “밈 주식 경계”vs“더 오른다”…‘서학개미 원픽’ 테슬라, 엇갈린 주가 전망   
1              "어라? 주가가 왜 이래"…'홈슬라' 기대하던 개미들 '멘붕'   
2             하나만 알고 둘은 모른다더니…금투세의 비밀 [정경준의 주식어때]   
3   "전세계 돈 몰린다, 이 주식 담아라"…전문가들 '깜짝 전망' [신민경의 테마록]   
4          '무료 환전 시대' 환테크로 돈 벌어볼까…환테크 넘어 환투기 조짐까지   
..                                            ...   
60        "석유 싣던 탱크로리에 식용유 운송" 중국서 경악한 사건, 주가도 폭삭   
61           한달 남짓한 MSCI 지수 변경일…편입·편출설 나오는 종목은 어디   
62     수익은 기본, 따박따박 돈 쌓이는 재미도…좋은 배당주 4가지 기준 [부꾸미]   
63                  기술주 투매 뉴욕증시, 은행 실적 주목하며 반등 출발   
64    [주간사모펀드] 한앤코, 4.7조 투자 펀드 조성…M&amp;A 주도권 다질까   

                                              summary  
0   테슬라 주가를 두고 엇갈린 전망이 나오고 있음. 테슬라 주가 강세의 배경으로는 월가...  
1   해상운임의 피크아웃(정점 통과) 우려 때문에 HMM 주가가 하락하고 있음. 해상 운...  
2   우리 주식시장이 저평가 받고 있는 이유는 원천적으로 우리 주식시장으로는 돈이 들어올...  
3   증권가에서 미국 빅테크주에 대한 전망이 긍정적임. 국내 빅테크주에 대해선 비관론이 ...  
4   엔화가 850원대까지 떨어지는 슈퍼 엔저 현상이 이어지며 환차익을 노리는 환테크 열...  
..                         

In [59]:
# csv로 저장
import os
os.chdir('C:/Users/myhyu/OneDrive/Desktop/2024 미래에셋/NewsData/')
summarized_df.to_csv('뉴스요약_ver2.csv', index=False)

## 예시

In [50]:
df['news_contents'][1]

'[\n"운임 급등에 \'흠슬라\' 재현되나 했더니"…목표가 하향 웬일? 하나증권 "지속불가능한 운임 강세…해운동맹 재편도 부담"삼성증권 "해운 성수기 앞두고 항만 적체 조짐"\n\n\n\nHMM 컨테이너선/ 한경DB국내 최대 컨테이너 선사인 HMM(옛 현대상선) 주가가 속절없이 하락하고 있다. 주가는 지난달 중순 이후 해상운임 급등에 힘입어 ‘흠슬라’(HMM+테슬라) 재현 기대와 함께 가파르게 상승했으나 이달 들어 돌연 꺾였다. 해상운임의 피크아웃(정점 통과) 우려 때문이다.주가 향배에 대한 증권가의 의견은 엇갈린다. 양대 운하 차질이 항만 적체 현상으로 이어지면서 운임 고공행진이 지속될 것이란 낙관론과 해상 운송서비스 공급 증가로 결국 해운업황이 꺾일 것이란 비관론이 동시에 나오고 있다. “홍해 사태 빼면 초과 공급…해운동맹 재편 따른 경쟁 격화 가능성도”한국거래소에 따르면 HMM은 이달 들어 지난 12일까지 12.07% 하락한 1만7560원에 거래를 마쳤다. 지난 2일 1만9970원으로 고점을 찍은 뒤 하루를 제외한 전 거래일 약세를 보였다.HMM 주가를 꺾은 건 해상운임의 상승 둔화에 따른 피크아웃 우려다. 해상 컨테이너 운임 지표인 상하이컨테이너운임지수(SCFI)는 지난 3월29일 이후 13주 연속 오름세를 유지하며 118.02% 상승했다. 다만 지난 5일 기준 해당 주간 상승폭은 전주 대비 0.52%에 그쳐 상승세가 둔화, 피크아웃 우려가 제기됐다.그동안은 이스라엘과 팔레스타인 무장정파 하마스 사이의 전쟁이 운임을 밀어 올렸다. 하마스를 지지하는 예멘의 후티반군이 홍해를 지나는 선박을 공격해 사실상 수에즈운하가 봉쇄되면서다. 수에즈운하를 통과하지 못하면 아시아와 유럽을 오가는 선박은 아프리카 대륙을 빙 돌아야 한다. 운항일정이 2주가량 더 걸린다. 한번 서비스하는 데 소요되는 시간이 늘어난 데 따라 운송서비스 공급량은 줄어든다. 공급 감소는 가격 상승으로 이어졌다.그러나 최근 이스라엘-팔레스타인 전쟁의 휴전 협상이 급물살을 타면서 운임 피크아웃 우려

비슷한 문장이 두 개인걸로 봐서... 약간 성능이 별로인듯  
여기서 튜닝해나가면 될 거 같음

In [47]:
summarized_df['summary'][1]

'해상운임의 피크아웃(정점 통과) 우려 때문에 HMM 주가가 하락하고 있음. 해상 운송서비스 공급 증가로 결국 해운업황이 꺾일 것이란 비관론이 동시에 나오고 있음. 해상운임의 상승 둔화에 따른 피크아웃 우려로 HMM의 주가가 하락함. 해상운임의 상승 둔화에 따른 피크아웃 우려와 해운동맹 재편에 따른 경쟁 격화 가능성도 있음. 2023~2025년 평균 선복량 증가율이 물동량 증가율을 초과하는 시기에 진입했다는 점이 근원적 우려임. 디얼라이언스 소속 해운사 중 가장 규모가 큰 독일의 하팍로이드가 내년 2월 디얼라이언스를 떠나 스위스 머스크와 협력하기로 하면서 디얼라이언스의 영업력이 쪼그라들 전망임. 중동 전쟁이 끝나도 환경 규제로 운임 강세가 이어질 것임. 해상 운임이 쉽게 꺾이지 않을 것이란 관측도 있음.'

In [53]:
summarized_df['summary'][3]

"증권가에서 미국 빅테크주에 대한 전망이 긍정적임. 국내 빅테크주에 대해선 비관론이 우세함. 미국 빅테크주와 국내 빅테크주 사이에서 개인 투자자들이 갈피를 못 잡고 있음. 국내 빅테크주보다는 '매그니피센트 7'에 베팅하는 분위기임. AI 시장은 '승자독식'이 강하게 작용하는 영역이어서 미국 빅테크 대비 국내 양대 빅테크의 경쟁력은 현저히 떨어진다고 봄. 미국에선 AI 붐이 일자 빅테크가 집중 조명을 받았지만 국내에선 빅테크가 아닌 반도체 종목들이 AI 수혜주로 꼽힘. 정책적 특수로 인해 밸류업(기업가치 제고) 관련주들로도 수급이 분산됨. 미국 전체 시가총액의 약 30%를 차지하는 '매그니피센트 7' 종목들은 미국 경제와 산업에 미치는 영향이 계속해서 커지고 있음."